### Описание

Пример взаимодействия с ПО MBAL черз программный интерфейс OpenServer

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import win32com.client as win32
from subprocess import Popen

### Настройки

In [3]:
# путь к исполняемому файлу MBAL
MbalPath = r"C:\Program Files (x86)\Petroleum Experts\IPM 7.5\mbal.exe"
# путь к файлу модели
InputFilename = r"D:\Appraisal\devon.mbi"
# систему единиц
UnitSystem = "Norwegian S.I."

### Вспомогательные функции

In [4]:
# функция для получения названия приложения из пакета IPM
def GetAppName(strval):
    pos = strval.find(".")
    if pos < 2:
        raise Exception("Badly formed tag string")
    appname = strval[0:pos]
    if appname != "GAP" and appname != "MBAL" and appname != "PROSPER":
        raise Exception("Unrecognised application name in tag string")
    return appname

In [5]:
# функция для выполнения команды
def DoCmd(server, cmd):
    err = server.DoCommand(cmd)
    if err > 0:
        err = server.GetErrorDescription(err)
        raise Exception(err)

In [6]:
# функция для задания значения переменной в приложении
def DoSet(server, varname, val):
    err = server.SetValue(varname, val)
    appname = GetAppName(varname)
    err = server.GetLastError(appname)
    if err > 0:
        err = server.GetErrorDescription(err)
        raise Exception(err)

In [7]:
# функция для чтения значения переменной из приложения
def DoGet(server, varname):
    val = server.GetValue(varname)
    appname = GetAppName(varname)
    err = server.GetLastError(appname)
    if err > 0:
        err = server.GetErrorDescription(err)
        raise Exception(err)
    return val

### Основной код

In [8]:
# запуск исполняемого файла MBAL
proc = Popen(MbalPath)

In [9]:
# подключение к программному интерфейсу MBAL
server = win32.Dispatch("PX32.OpenServer.1")

In [10]:
# вызов команды открытия файла модели MBAL
DoCmd(server, "MBAL.OPENFILE(\"" + InputFilename + "\"")

In [11]:
# вызов команды установки системы единиц
DoCmd(server, "MBAL.SETUNITSYS(\"" + UnitSystem + "\")")

In [12]:
# чтение количества танков в таблице результатов расчета
tankcnt = int(DoGet(server, "MBAL.MB.TRES.COUNT"))
print("Количество танков:", tankcnt)

Количество танков: 3


In [13]:
df_list = {}

In [14]:
# чтение расчетных параметров из таблицы результатов по каждому танку
for i in range(tankcnt):
    df = pd.DataFrame(columns=["DATE", "GAS"], dtype=str)
    prefix = "MBAL.MB.TRES[{Prediction}][" + str(i+1) + "]"
    name = DoGet(server, prefix + ".NAME")
    rowcnt = int(DoGet(server, prefix + ".COUNT"))
    for j in range(rowcnt):
        date = DoGet(server, prefix + "[" + str(j) + "].TIME")
        gas = DoGet(server, prefix + "[" + str(j) + "].AVEGASTOTAL")
        #df = df.append(pd.DataFrame([[date, gas]], columns=["DATE", "GAS"]))
        df.loc[j] = [date, gas]
    df["DATE"] = df["DATE"].astype('datetime64[ns]')
    df["GAS"] = df["GAS"].astype('float64')
    df.loc[df["GAS"] > 10000000000, "GAS"] = np.NaN
    df_list[name] = df

In [15]:
# закрытие программного интерфейса
server = None

In [16]:
# завершение работы процесса MBAL
proc.terminate()

In [17]:
# вывод результатов на графике
fig, ax = plt.subplots()
for name in df_list:
    df = df_list[name]
    df.plot(x="DATE", y="GAS", label=name, ax=ax)
ax.set_xlabel("Year")
ax.set_ylabel("Gas Rate")
plt.show()